In [0]:
from pyspark.sql import functions as F

In [0]:
source = dbutils.widgets.get("source")
table = dbutils.widgets.get("table")

In [0]:
df = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format", "csv")\
            .option("cloudFiles.schemaLocation", f"abfss://bronze@storageawesum.dfs.core.windows.net/{source}/{table}/_schema")\
            .option("cloudFiles.schemaEvolutionMode", "addNewColumns")\
            .load(f"abfss://raw@storageawesum.dfs.core.windows.net/staging/{source}/{table}/*/")

# Extract the full file path (input_file_name gives complete abfss path)
df = df.withColumn("file_path", F.input_file_name())\
    .withColumn("extract_timestamp", F.concat(F.lit("DH_"+table.upper()+"_"), F.regexp_extract("file_path", fr"{table}_(\d{{8}}_\d{{6}})", 1)))\
    .withColumn("dbx_extract_timestamp", F.current_timestamp())\
    .drop(F.col("file_path")).drop(F.col("_rescued_data"))


df.writeStream\
    .format("delta")\
    .option("checkpointLocation", f"abfss://bronze@storageawesum.dfs.core.windows.net/{source}/{table}/_checkpoint")\
    .partitionBy("extract_timestamp")\
    .option("mergeSchema", "true")\
    .trigger(availableNow=True)\
    .start(f"abfss://bronze@storageawesum.dfs.core.windows.net/{source}/{table}/")